In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

0.6065030097961426

In [3]:
# casos = casos_confirmados.fix_mun_resid_casos('mun_atend')
# obitos = casos_confirmados.fix_mun_resid_obitos()
casos = casos_confirmados.get_casos()
obitos = casos_confirmados.get_obitos()

# obitos.to_excel('novos_obitos.xlsx')

print(f"{len(casos)};{len(obitos)}")
obitos.loc[~obitos['hash'].isin(casos['hash'])]

370775;7213


,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,hash,hash_less,hash_more,hash_idade


In [4]:
obitos = obitos.rename(columns={'data_do_obito': 'data_obito'})
obitos['obito'] = 'SIM'

casos = pd.merge(left=casos, right=obitos[['obito','data_obito','hash']], on='hash', how='left')

casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'data_obito'] = pd.NaT

print(f"{len(casos)};{len(casos.loc[casos['obito']=='SIM'])}")

370775;7213


In [5]:
casos['excluir'] = 'NAO'

print(f"Duplicados com a mesma idade: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
print(f"Duplicados com idade menos 1: {casos.loc[casos['hash_less'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_less'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_less'].isin(casos['hash']), 'hash_less']
print(f"Duplicados com idade mais 1: {casos.loc[casos['hash_more'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_more'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_more'].isin(casos['hash']), 'hash_more']
print(f"todos os duplicados: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
duplicados = casos.loc[casos.duplicated('hash', keep=False)]
print(f"Valores unicos: {len(duplicados.groupby('hash'))}")

Duplicados com a mesma idade: (947, 18)
Duplicados com idade menos 1: (1937, 18)
Duplicados com idade mais 1: (4, 18)
todos os duplicados: (4811, 18)
Valores unicos: 2403


In [6]:
def get_better(df):
    scores = np.zeros(len(df))

    for i, serie in enumerate(df.iterrows()):
        _, row = serie

        if row['obito'] == 'SIM':
            scores[i] = 1000

    i = np.argmax(scores)
    return df.iloc[i].name

In [7]:
manter = []
for hash, group in duplicados.groupby('hash'):
    idx = get_better(group)
    manter.append(idx)

In [8]:
duplicados = set(duplicados.index.tolist())
duplicados = duplicados - set(manter)

casos.loc[duplicados,'excluir'] = 'SIM'
casos.loc[duplicados,'hash'] = None
casos.loc[duplicados,'hash_less'] = None
casos.loc[duplicados,'hash_more'] = None

In [9]:
new_casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir','hash']]
new_obitos = obitos[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito','hash']]

In [10]:
writer = pd.ExcelWriter(join(output,"novos_casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()